In [4]:
library(h2o)
options(warnings=-1)

In [5]:
h2o.init(nthreads = -1)

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         4 seconds 356 milliseconds 
    H2O cluster timezone:       Asia/Kolkata 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.20.0.2 
    H2O cluster version age:    12 days  
    H2O cluster name:           H2O_started_from_R_raj_cjn618 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   3.56 GB 
    H2O cluster total cores:    8 
    H2O cluster allowed cores:  8 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.4.3 (2017-11-30) 



In [6]:
url <-"http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip"
data=h2o.importFile(url)

  |======================================================================| 100%


In [7]:
parts=h2o.splitFrame(data,c(0.8,0.1),seed=69)
train<-parts[[1]]
valid<-parts[[2]]
test<-parts[[3]]

In [8]:
y <-'IsArrDelayed' # binomial Classification
ignoreFields=c('ArrDelay',
 'DepDelay','CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay','IsArrDelayed',
 'IsDepDelayed','ActualElapsedTime','ArrTime')
x2<-setdiff(colnames(data),ignoreFields)

In [9]:
system.time(g<- h2o.grid("deeplearning",search_criteria = list(strategy="RandomDiscrete",max_models=4 ),# When testing with epochs 0.01
                                                  #max_models=12
             hyper_params=list(seed=c(77),#Placeholder
                               l1=c(0,1e-6,3e-6,1e-5),l2=c(0,1e-6,3e-6,1e-5),input_dropout_ratio=c(0,0.1,0.2,0.3),
                              hidden_dropout_ratios=list(
                                  c(0,0),c(0.2,0.2),c(0.4,0.4),c(0.6,0.6)
                              )
                            ),
                               grid_id="dl_test",
                               x=x2,
                               y=y,
                               hidden=c(400,400),
                               epochs=0.01,#to test it quickly
                               training_frame=train,
                               validation_frame=valid,
                               activation="RectifierWithDropout"))

             
                               

  |======================================================================| 100%


   user  system elapsed 
  1.078   0.078 123.744 

In [10]:
g

H2O Grid Details

Grid ID: dl_test 
Used hyper parameters: 
  -  hidden_dropout_ratios 
  -  input_dropout_ratio 
  -  l1 
  -  l2 
  -  seed 
Number of models: 4 
Number of failed models: 0 

Hyper-Parameter Search Summary: ordered by increasing logloss
  hidden_dropout_ratios input_dropout_ratio     l1     l2 seed       model_ids
1            [0.6, 0.6]                 0.0 1.0E-6    0.0   77 dl_test_model_2
2            [0.6, 0.6]                 0.3 1.0E-5 1.0E-6   77 dl_test_model_0
3            [0.0, 0.0]                 0.3 3.0E-6 1.0E-5   77 dl_test_model_3
4            [0.6, 0.6]                 0.1 1.0E-5 3.0E-6   77 dl_test_model_1
             logloss
1 0.7471338356765426
2 1.0276765483506496
3 1.2854761101888248
4 1.5711927225839926

In [11]:
system.time(g2<- h2o.grid("deeplearning",search_criteria = list(strategy="RandomDiscrete",max_models=12),# When testing with epochs 0.01
                                                  #max_models=4
             hyper_params=list(seed=c(77),#Placeholder
                               l1=c(0,1e-6,3e-6,1e-5),l2=c(0,1e-6,3e-6,1e-5),input_dropout_ratio=c(0,0.1,0.2,0.3),
                              hidden_dropout_ratios=list(
                                  c(0,0),c(0.2,0.2),c(0.4,0.4),c(0.6,0.6)
                              )
                            ),
                               grid_id="dlB",
                               x=x2,
                               y=y,
                               hidden=c(400,400),
                               epochs=40,#Starting to overfit
                               training_frame=train,
                               validation_frame=valid,
                               activation="RectifierWithDropout"))

  |======================================================================| 100%


    user   system  elapsed 
  53.946    3.612 7136.607 

In [12]:
g2

H2O Grid Details

Grid ID: dlB 
Used hyper parameters: 
  -  hidden_dropout_ratios 
  -  input_dropout_ratio 
  -  l1 
  -  l2 
  -  seed 
Number of models: 12 
Number of failed models: 0 

Hyper-Parameter Search Summary: ordered by increasing logloss
   hidden_dropout_ratios input_dropout_ratio     l1     l2 seed    model_ids
1             [0.6, 0.6]                 0.0 1.0E-6 1.0E-6   77  dlB_model_2
2             [0.0, 0.0]                 0.0 1.0E-5 1.0E-5   77  dlB_model_5
3             [0.2, 0.2]                 0.0 3.0E-6 1.0E-5   77  dlB_model_3
4             [0.4, 0.4]                 0.0 3.0E-6 3.0E-6   77 dlB_model_10
5             [0.4, 0.4]                 0.1 1.0E-6 1.0E-6   77  dlB_model_4
6             [0.2, 0.2]                 0.1 1.0E-6    0.0   77  dlB_model_8
7             [0.4, 0.4]                 0.1 1.0E-6 1.0E-5   77 dlB_model_11
8             [0.0, 0.0]                 0.2 1.0E-6 1.0E-5   77  dlB_model_7
9             [0.4, 0.4]                 0.2 1.0E-6 1.0

Quite a bit of noise in above logloss, Drop 0.6 from hidden dropout ratio and 0.2 and 0.3 for input dropout ratio and make eight more models

Remember : Keep the same grid_id, new and old get merged

In [14]:
system.time(g2<- h2o.grid("deeplearning",search_criteria = list(strategy="RandomDiscrete",max_models=8 ),# When testing with epochs 0.01
                                                  #max_models=12
             hyper_params=list(seed=c(88),#Different value just so we can see new one
                               l1=c(0,1e-6,3e-6,1e-5),l2=c(0,1e-6,3e-6,1e-5),input_dropout_ratio=c(0,0.1),
                              hidden_dropout_ratios=list(
                                  c(0,0),c(0.2,0.2),c(0.4,0.4)
                              )
                            ),
                               grid_id="dlB",
                               x=x2,
                               y=y,
                               hidden=c(400,400),
                               epochs=40,#Starting to overfit
                               training_frame=train,
                               validation_frame=valid,
                               activation="RectifierWithDropout"))

  |======================================================================| 100%


    user   system  elapsed 
  37.341    2.553 4999.230 

In [15]:
g2

H2O Grid Details

Grid ID: dlB 
Used hyper parameters: 
  -  hidden_dropout_ratios 
  -  input_dropout_ratio 
  -  l1 
  -  l2 
  -  seed 
Number of models: 20 
Number of failed models: 0 

Hyper-Parameter Search Summary: ordered by increasing logloss
   hidden_dropout_ratios input_dropout_ratio     l1     l2 seed    model_ids
1             [0.6, 0.6]                 0.0 1.0E-6 1.0E-6   77  dlB_model_2
2             [0.0, 0.0]                 0.0 1.0E-5    0.0   88 dlB_model_13
3             [0.0, 0.0]                 0.0 1.0E-5 1.0E-5   77  dlB_model_5
4             [0.2, 0.2]                 0.0 3.0E-6 1.0E-5   77  dlB_model_3
5             [0.2, 0.2]                 0.0    0.0 1.0E-6   88 dlB_model_18
6             [0.4, 0.4]                 0.0 1.0E-6 1.0E-6   88 dlB_model_16
7             [0.4, 0.4]                 0.0 3.0E-6 3.0E-6   88 dlB_model_17
8             [0.4, 0.4]                 0.0 3.0E-6 3.0E-6   77 dlB_model_10
9             [0.0, 0.0]                 0.0    0.0 1.0

In [ ]:
best_model <- h2o.getModel(g@model_ids[[1]]) #ist models is th best one with least log loss
h2o.saveModel(best_model,"/tmp/")
#validation and test errors are roughly the same which is a good sign
h2o.performance(best_model,valid= TRUE)
h2o.performance(best_model,test)



[1] "/private/tmp/dl_test_model_2"

H2OBinomialMetrics: deeplearning
** Reported on validation data. **
** Metrics reported on full validation frame **

MSE:  0.2737503
RMSE:  0.5232115
LogLoss:  0.7471338
Mean Per-Class Error:  0.5
AUC:  0.4758771
Gini:  -0.04824578

Confusion Matrix (vertical: actual; across: predicted) for F1-optimal threshold:
       NO  YES    Error        Rate
NO      0 1933 1.000000  =1933/1933
YES     0 2339 0.000000     =0/2339
Totals  0 4272 0.452481  =1933/4272

Maximum Metrics: Maximum metrics at their respective thresholds
                        metric threshold    value idx
1                       max f1  0.455355 0.707609 399
2                       max f2  0.455355 0.858160 399
3                 max f0point5  0.455355 0.601997 399
4                 max accuracy  0.455355 0.547519 399
5                max precision  0.821339 1.000000   0
6                   max recall  0.455355 1.000000 399
7              max specificity  0.821339 1.000000   0
8             max absolute_mcc  0.576630 0.13